## Setup

In [59]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=6)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import time_solution

## Part a

### Brute Force Approach
Let's start with a simple brute force approach that checks all values of `t` to find the valid solutions for each `(max_time, distance_goal)` pair.

- After accelerating for time `t`, our speed is `v = t`
- Distance traveled `d = v * T_remaining =  t * (max_time - t)`

So we can iterate over all possible `t` values from `1` to `max_time - 1`, and count how many times the distance traveled exceeds `distance_goal`. Finally, we take the product of these counts for all input pairs.

As a one-liner:
```python
prod(
    sum(1 if t * (t_max - t) > d else 0 for t in range(1, t_max))
    for t_max, d in [*zip(*[[*map(int, line.split()[1:])] for line in input_data.splitlines()], strict=True)]
)
```


In [61]:
# Imports
from math import prod

In [ ]:
# Functions
def parse_input_a(input_data: str) -> list[tuple[int, int]]:
    """Parse input data to a list of (max_time, distance_goal) pairs."""
    return [*zip(*[[*map(int, line.split()[1:])] for line in input_data.splitlines()], strict=True)]


def count_valid_t_brute_force(max_time: int, distance_goal: int) -> int:
    """Count the number of 't' that result in distance > distance_goal using brute force."""
    return sum(1 if t * (max_time - t) > distance_goal else 0 for t in range(1, max_time))


def solve_a_brute_force(input_data: str) -> int:
    """Find the product of the valid t count for each (max_time, distance_goal) pair using brute force."""
    return prod(
        count_valid_t_brute_force(max_time, distance_goal) for max_time, distance_goal in parse_input_a(input_data)
    )

In [63]:
# Correctness check
assert str(solve_a_brute_force(example.input_data)) == example.answer_a

In [ ]:
# Performance check
time_a_brute_force = time_solution(solve_a_brute_force, input_data, iterations=500, runs=10, time_unit="us")
print(f"Brute force approach takes {time_a_brute_force:.2f} μs")

Brute force approach takes 12.55 μs


### Analytical approach
As we saw above, `d = t * (max_time - t) = t * max_time - t^2`. We want `t` such that `d > distance_goal`, i.e. such that `-t^2 + max_time*t - distance_goal > 0`.

Valid `t` values lie between the two roots of `-t^2 + max_time * t - distance_goal = 0`, which we can find using the quadratic formula.

The inequality holds for `t` in the open interval `(r1, r2)`, where `r1` ≤ `r2`. Thus, the number of valid integer `t` can be expressed as `valid_t_count = ceil(r2) - floor(r1) - 1`.


In [161]:
# Imports
from math import sqrt

In [ ]:
# Functions
def count_valid_t_analytical(max_time: int, distance_goal: int) -> int:
    """Count the number of integer `t` between the roots of -t^2 + max_time -distance_goal"."""
    a = -1
    b = max_time
    c = -distance_goal

    d = b**2 - 4 * a * c
    r1 = (-b + sqrt(d)) / (2 * a)
    r2 = (-b - sqrt(d)) / (2 * a)
    # Count the integers strictly between r1 and r2
    return int(r2 // 1 - r1 // 1) - (1 if r2 % 1 == 0 else 0)


def solve_a_analytical(input_data: str) -> int:
    """Find the product of the valid t count for each (max_time, distance_goal) pair using the quadratic formula."""
    return prod(
        count_valid_t_analytical(max_time, distance_goal) for max_time, distance_goal in parse_input_a(input_data)
    )

In [178]:
# Correctness check
assert str(solve_a_analytical(example.input_data)) == example.answer_a

In [ ]:
# Performance check
time_a_analytical = time_solution(solve_a_analytical, input_data, iterations=500, runs=10, time_unit="us")
print(
    f"Analytical approach takes {time_a_analytical:.2f} μs"
    f", which is {time_a_brute_force / time_a_analytical:.2f}x faster than brute force."
)

Analytical approach takes 3.72 μs, which is 3.37x faster than brute force.


In [ ]:
# Submit answer
puzzle.answer_a = solve_a_analytical(input_data)

That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


## Part b

### Brute force approach
Using the brute-force approach will take some time but should work.

In [ ]:
# Functions
def parse_input_b(input_data: str) -> list[int]:
    """Parse input data to one large (max_time, distance_goal) pair."""
    return [int(line.split(":")[1].replace(" ", "")) for line in input_data.splitlines()]


def solve_b_brute_force(input_data: str) -> int:
    """Find the valid t count for the large (max_time, distance_goal) pair using brute force."""
    return count_valid_t_brute_force(*(parse_input_b(input_data)))

In [174]:
# Correctness check
assert str(solve_b_brute_force(example.input_data)) == example.answer_b

In [ ]:
# Performance check
time_b_brute_force = time_solution(solve_b_brute_force, input_data, iterations=3, runs=3, time_unit="s")
print(f"Brute force approach takes {time_b_brute_force:.2f} s")

Brute force approach takes 2.56 s


### Analytical approach
Using the analytical approach from part A should be a lot faster here.


In [180]:
# Functions
def solve_b_analytical(input_data: str) -> int:
    """Find the valid t count for the large (max_time, distance_goal) pair using the quadratic formula."""
    return count_valid_t_analytical(*(parse_input_b(input_data)))

In [181]:
# Correctness check
assert str(solve_b_analytical(example.input_data)) == example.answer_b

In [ ]:
# Performance check
time_b_analytical = time_solution(solve_b_analytical, input_data, iterations=500, runs=10, time_unit="us")
print(
    f"Quadratic formula approach takes {time_b_analytical:.2f} μs"
    f", which is {1_000_000 * time_b_brute_force / time_b_analytical:,.0f} x faster than brute force."
)

Quadratic formula approach takes 1.28 μs, which is 2,001,287 x faster than brute force.


That's a huge speedup, I don't think i've even seen an improvement of a factor over a million in an AoC optimization before!

In [ ]:
# Submit answer
puzzle.answer_b = solve_b_analytical(input_data)

That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 6! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


### Bonus: solving part A using the quadratic formula
We can also solve part A using the quadratic formula by finding the maximum distance traveled for each max. Let's see if that speeds things up.

In [ ]:
# Functions
def solve_a_quadratic(input_data: str) -> int:
    """Solve part A of the puzzle using the quadratic formula."""
    return prod(
        count_valid_t_analytical(max_time, distance_goal)
        for max_time, distance_goal in [
            *zip(*[[*map(int, line.split()[1:])] for line in input_data.splitlines()], strict=True)
        ]
    )

In [145]:
# Correctness check
assert str(solve_a_quadratic(example.input_data)) == example.answer_a

In [ ]:
# Performance check
time_a_quadratic = time_solution(solve_a_quadratic, input_data, iterations=500, runs=10, time_unit="us")
print(
    f"Quadratic formula approach for part A takes {time_a_quadratic:.2f} μs"
    f", which is {time_a_brute_force / time_a_quadratic:.2f} x faster than the brute-force implementation."
)

Quadratic formula approach for part A takes 3.88 μs, which is 16.11 x faster than the original implementation.
